In [0]:
# commenting the access key in order to publsih this notebook to GITHUB
spark.conf.set(
    "fs.azure.account.key.goodreadsreviews60104758.dfs.core.windows.net",
    "<accesskey>"
)

In [0]:
# Load the books dataset from the silver layer
books = spark.read.parquet(
"abfss://lakehouse@goodreadsreviews60104758.dfs.core.windows.net/processed/books/"
)
# Load the authors dataset from the silver layer
authors = spark.read.parquet(
"abfss://lakehouse@goodreadsreviews60104758.dfs.core.windows.net/processed/authors/"
)
# Display the first few records to confirm the data was loaded correctly
books.show(5)
authors.show(5)
# Display the columns and their data types to verify the schema
books.printSchema()
authors.printSchema()


+----------+------------------+------------+-------------+----------+--------+--------------+-----------+--------------------+---------+--------------------+--------------------+---------+---------------+-------------+-----------------+-------------------+----------------+--------------------+--------------------+-------+-------------+-------+--------------------+--------------------+
|      isbn|text_reviews_count|country_code|language_code|      asin|is_ebook|average_rating|kindle_asin|         description|   format|                link|           publisher|num_pages|publication_day|       isbn13|publication_month|edition_information|publication_year|                 url|           image_url|book_id|ratings_count|work_id|               title|title_without_series|
+----------+------------------+------------+-------------+----------+--------+--------------+-----------+--------------------+---------+--------------------+--------------------+---------+---------------+-------------+------

In [0]:

from pyspark.sql.functions import col, length, trim, count, when
reviews = spark.read.parquet("abfss://lakehouse@goodreadsreviews60104758.dfs.core.windows.net/processed/reviews/")
reviews.show(5, truncate=False)
reviews.printSchema()
total_rows = reviews.count()
null_review_id = reviews.filter(col("review_id").isNull()).count()
null_book_id = reviews.filter(col("book_id").isNull()).count()
null_user_id = reviews.filter(col("user_id").isNull()).count()
null_rating = reviews.filter(col("rating").isNull()).count()
empty_text = reviews.filter( (col("review_text").isNull()) | (trim(col("review_text")) =="") ).count()
print(f"Total rows: {total_rows}")
print(f"NULL review_id: {null_review_id}, NULL book_id: {null_book_id}, NULL user_id:{null_user_id}, NULL rating: {null_rating}")
print(f"Empty/NULL review_text: {empty_text}")

+--------------------------------+--------+--------------------------------+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------------------------------+
|review_id                       |book_id |user_id                         |rating|review_text                                                                                                                                                                  

In [0]:
from pyspark.sql.functions import col, trim, length
# Start from the existing Parquet-loaded DataFrame
# (Assumes you already did: reviews = spark.read.parquet(".../processed/reviews/"))
df = reviews
# 1) Drop rows missing critical keys
df = df.filter(
col("review_id").isNotNull() &
col("book_id").isNotNull() &
col("user_id").isNotNull()
)
# 2) Enforce rating to be integer in [1..5]
df = df.withColumn("rating_int", col("rating").cast("int"))
df = df.filter(
col("rating_int").isNotNull() &
(col("rating_int") >= 1) &
(col("rating_int") <= 5)
)
# 3) Normalize text; drop empty or ultra-short reviews (<10 chars after trim)
df = df.withColumn("review_text", trim(col("review_text")))
df = df.filter(
col("review_text").isNotNull() &
(length(col("review_text")) >= 10)
)
# 4) De-duplicate by review_id (keep arbitrary first; refine if you have timestamps)
df = df.dropDuplicates(["review_id"])
# 5) Select final shape
reviews_clean_v1 = df.select(
"review_id",
"book_id",
"user_id",
col("rating_int").alias("rating"),
"review_text",
"n_votes",
"date_added"
)

In [0]:
# Write the cleaned reviews back to the silver layer (overwrite)
reviews_clean_v1.write.mode("overwrite").parquet(
"abfss://lakehouse@goodreadsreviews60104758.dfs.core.windows.net/processed/reviews/"
)
# Sanity check: re-read from disk and inspect schema and a few rows
reviews_verified = spark.read.parquet(
"abfss://lakehouse@goodreadsreviews60104758.dfs.core.windows.net/processed/reviews/"
)
reviews_verified.printSchema()
reviews_verified.show(5, truncate=False)
print(f"Written cleaned rows: {reviews_verified.count()}")

root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- date_added: string (nullable = true)

+--------------------------------+--------+--------------------------------+------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

**HOMEWORK PART 1**

In [0]:
silver_base = "abfss://lakehouse@goodreadsreviews60104758.dfs.core.windows.net/processed"
books_path = f"{silver_base}/books/"
authors_path = f"{silver_base}/authors/"
reviews_path = f"{silver_base}/reviews/"
book_authors_path = f"{silver_base}/book_authors/"
book_authors_rejects_path = f"{silver_base}/book_authors_rejects/"


In [0]:
books = spark.read.parquet(books_path)
authors = spark.read.parquet(authors_path)
reviews_verified = spark.read.parquet(reviews_path)

books.printSchema()
books.show(5, truncate=False)

authors.printSchema()
authors.show(5, truncate=False)

reviews_verified.printSchema()
reviews_verified.show(5, truncate=False)


root
 |-- isbn: string (nullable = true)
 |-- text_reviews_count: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- is_ebook: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- kindle_asin: string (nullable = true)
 |-- description: string (nullable = true)
 |-- format: string (nullable = true)
 |-- link: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- num_pages: string (nullable = true)
 |-- publication_day: string (nullable = true)
 |-- isbn13: string (nullable = true)
 |-- publication_month: string (nullable = true)
 |-- edition_information: string (nullable = true)
 |-- publication_year: string (nullable = true)
 |-- url: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- ratings_count: string (nullable = true)
 |-- work_id: string (nullable = true)
 |-- title: string (null

In [0]:
books.select("book_id").show(5)
print("Columns:", books.columns)


+-------+
|book_id|
+-------+
|5333265|
|1333909|
|7327624|
|6066819|
| 287140|
+-------+
only showing top 5 rows
Columns: ['isbn', 'text_reviews_count', 'country_code', 'language_code', 'asin', 'is_ebook', 'average_rating', 'kindle_asin', 'description', 'format', 'link', 'publisher', 'num_pages', 'publication_day', 'isbn13', 'publication_month', 'edition_information', 'publication_year', 'url', 'image_url', 'book_id', 'ratings_count', 'work_id', 'title', 'title_without_series']


In [0]:
# Join reviews with books on book_id
rev_books = reviews_verified.join(books, on="book_id", how="left")

print("Joined reviews + books:")
rev_books.select("book_id", "title", "review_id", "user_id", "rating").show(5, truncate=False)


Joined reviews + books:
+--------+------------------------------------------------------------------------------------------------+--------------------------------+--------------------------------+------+
|book_id |title                                                                                           |review_id                       |user_id                         |rating|
+--------+------------------------------------------------------------------------------------------------+--------------------------------+--------------------------------+------+
|21010103|Push (Push, #1)                                                                                 |4a87dd092b17626e4c7ff591ed28956a|01ec1a320ffded6b2dd47833f2c8e4fb|4     |
|577097  |Jim & Casper Go to Church: Frank Conversation about Faith, Churches, and Well-Meaning Christians|913000a1a3cf638df10b2b5f24d6698f|84f81d98fa16544c4ea9b41ea1157728|3     |
|26031154|Lowriders to the Center of the Earth (Book 2)                

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

book_authors = books.select("book_id").withColumn("author_id", (monotonically_increasing_id() % authors.count()))
book_authors.show(5)
print("Bridge table created with artificial mapping:")
book_authors.printSchema()



+-------+---------+
|book_id|author_id|
+-------+---------+
|5333265|        0|
|1333909|        1|
|7327624|        2|
|6066819|        3|
| 287140|        4|
+-------+---------+
only showing top 5 rows
Bridge table created with artificial mapping:
root
 |-- book_id: string (nullable = true)
 |-- author_id: long (nullable = true)



In [0]:
from pyspark.sql import functions as F, Window

# creating gold path
base = "abfss://lakehouse@goodreadsreviews60104758.dfs.core.windows.net"
gold_path_v1 = f"{base}/Gold_layer/curated_reviews_delta/"

books_sel = books.select(
    "book_id",
    "title",
    "language_code"
)

authors_idx = (
    authors.select("author_id").distinct()
           .withColumn("idx", F.row_number().over(Window.orderBy(F.col("author_id").cast("long"))) - 1)
)
author_cnt = authors_idx.count()

book_idx = (
    books.select("book_id")
         .withColumn("idx", (F.abs(F.hash("book_id")) % F.lit(author_cnt)).cast("long"))
)

book_authors = (
    book_idx.join(authors_idx, "idx", "left")
            .select("book_id", "author_id")
)

df_gold_v1 = (
    reviews_clean_v1
        .join(books_sel, "book_id", "left")
        .join(book_authors, "book_id", "left")
        .join(authors.select("author_id", "name"), "author_id", "left")
        .select(
            "review_id",
            "book_id",
            "title",
            "author_id",
            "name",
            "user_id",
            "rating",
            "review_text",
            "language_code",
            "n_votes",
            "date_added"
        )
)


In [0]:
df_gold_v1.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(gold_path_v1)

df = spark.read.format("delta").load(gold_path_v1)
df.printSchema()
df.show(10, truncate=False)

root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- date_added: string (nullable = true)

+--------------------------------+--------+-----------------------------------------------------------+---------+---------------------------+--------------------------------+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
spark.sql(f"CREATE TABLE IF NOT EXISTS curated_reviews USING DELTA LOCATION '{gold_path_v1}'")


DataFrame[]

In [0]:
# Use Spark SQL to query the Delta table
spark.sql(
    """
    SELECT
        review_id,
        book_id,
        title,
        author_id,
        name,
        user_id,
        rating
    FROM curated_reviews
    LIMIT 10
"""
).show(
    truncate=50
)  # truncate review text for readability

+--------------------------------+--------+--------------------------------------------------+---------+----------------+--------------------------------+------+
|                       review_id| book_id|                                             title|author_id|            name|                         user_id|rating|
+--------------------------------+--------+--------------------------------------------------+---------+----------------+--------------------------------+------+
|3818908d5a98733ba8e0d9340aaa01d3|17282103|                                  هكذا تكلم زرادشت|   196327|      Jody Rosen|b72b9ef6b43e415ee373a737ff3c43d1|     3|
|a04023de2a39a2cabfb956e56cb752e6|   35350|                           What the Body Remembers|    35285| Jimmy McDonough|b9c1edf6bcc9b18198bbedf212c1cee5|     3|
|092beaa2de4ea5a773fbeef45b710a2c|   92146|                    Carved in Bone (Body Farm, #1)|    88874|     Cris Freddi|6e1d7ac0a6e738b23864736a30957eaf|     5|
|ba230623304ed18eed0e93b96ac

In [0]:
spark.sql("SELECT COUNT(*) FROM curated_reviews").show()
spark.sql("SELECT * FROM curated_reviews LIMIT 5").show(truncate=True)

+--------+
|count(1)|
+--------+
|  788770|
+--------+

+--------------------+--------+--------------------+---------+---------------+--------------------+------+--------------------+-------------+-------+--------------------+
|           review_id| book_id|               title|author_id|           name|             user_id|rating|         review_text|language_code|n_votes|          date_added|
+--------------------+--------+--------------------+---------+---------------+--------------------+------+--------------------+-------------+-------+--------------------+
|3818908d5a98733ba...|17282103|    هكذا تكلم زرادشت|   196327|     Jody Rosen|b72b9ef6b43e415ee...|     3|hkdh tklm zrdsht ...|          ara|      3|Sat Feb 09 11:01:...|
|a04023de2a39a2cab...|   35350|What the Body Rem...|    35285|Jimmy McDonough|b9c1edf6bcc9b1819...|     3|Read this for a c...|          eng|      0|Mon Sep 23 10:23:...|
|092beaa2de4ea5a77...|   92146|Carved in Bone (B...|    88874|    Cris Freddi|6e1d7ac0a6e